<a href="https://colab.research.google.com/github/davidkuernsteiner/numerical_optimization/blob/main/phase2_conjugate_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3 style="background-color: #D3D92B;"><br>Numerical Optimisation. Project 1<br></h3><br>

<h3 style="background-color: #D3D92B;"><br>Team Information<br></h3><br>

<i>Group 1<br>
Participants information in alphabetical order</i>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">David</td>
    <td style = "text-align: left">Kürnsteiner</td>
    <td style = "text-align: left">11820336</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Elias</td>
    <td style = "text-align: left">Ramoser</td>
    <td style = "text-align: left">11918558</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Georg</td>
    <td style = "text-align: left">Storz</td>
    <td style = "text-align: left">11918811</td>
    </tr>
  
</table>


<h3 style="background-color: #D3D92B;"><br>Implementation<br></h3><br>
<i>All points x are represented as numpy arrays. Function f returns a scalar with grad_f and hessian_f returning numpy arrays.<i>
<hr><h4 style="background-color: #ADB8FF;"><br>Imports<br></h4><br>
<i>Describe how to install additional packages, if you have some, here</i>


In [ ]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from numpy.linalg import *
from sklearn.datasets import make_spd_matrix

<hr><h4 style="background-color: #ADB8FF;"><br>Stopping criteria<br></h4><br>
<i>Function returns True if the gradient of f at xk relative to x0 is smaller than parameter tol. Additionally there is an upper bound for iterations to stop non converging algorithms.<i>

In [ ]:
def stop_crit(grad_f, xk, x0, i, tol=1e-8, max_iter=5000):
  if i > max_iter: 
    return True
  elif norm(grad_f(xk)) <= tol * norm(grad_f(x0)):
    return True
  return False

<hr><h4 style="background-color: #ADB8FF;"><br>Varibales scaling<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.<i>

<hr><h4 style="background-color: #ADB8FF;"><br>Stabilising algorithm<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.<i>

In [ ]:
#your function for stabilising goes here

<hr><h4 style="background-color: #ADB8FF;"><br>Fighting floating-point numbers and roundoff error<br></h4><br>
<i>Place your reasoning, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.<i>

<hr><h4 style="background-color: #ADB8FF;"><br>Inverting matrices<br></h4><br>
<i>linear_solve() provides a way to solve linear systems of equations using a LU-factorization of A and subsequent forward and backward substitution as described in the book. This solver proves to be quite unstable though in practical applications. We therefore use the numpy implementation of solve(). <i>

In [ ]:
def forward_substitution(L, b):

    n = L.shape[0]

    y = np.zeros_like(b, dtype=np.double);

    y[0] = b[0] / L[0, 0]

    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    
    n = U.shape[0]

    x = np.zeros_like(y, dtype=np.double);

    x[-1] = y[-1] / U[-1, -1]
    
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

def linear_solve(A, b):
    
    P, L, U = scipy.linalg.lu(A)
    
    y = forward_substitution(L, P @ b)
    
    return back_substitution(U, y)

<hr><h4 style="background-color: #ADB8FF;"><br>Gradients calculation<br></h4><br>
<i>Following functions are wrapper functions that provide approximations of the gradient and hessian of f using the forward-difference approach as described in the book. <i>

In [ ]:
def e_i(size, index):
  arr = np.zeros(size)
  arr[index] = 1.0
  return arr

def approx_grad(f, e=1.1*10**-8):
  def grad_f(x):
    if x.size == 1:
      return (f(x + e) - f(x)) / e
    return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])
  return grad_f

def approx_hessian(f, e=1.1*10**-8):
  def hessian_f(x):
    if x.size == 1:
      return (f(x + 2*e) - 2*f(x + e) + f(x)) / e**2
    return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
                      x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
                      x)) / e**2 for j in range(x.size)] for i in range(x.size)])
  return hessian_f

<hr><h4 style="background-color: #ADB8FF;"><br>Additional objects you implemented<br></h4><br>
<i>The class Problem() provides an object to generate and set up quadratic and non some non quadratic test problems for the algorithms with.<i>

In [ ]:
class Problem():

  def __init__(self):

    self.f = None
    self.grad_f = None
    self.hessian_f = None
    self.min_x = None

  def quadratic(self, n_dim, rseed):

    rng = np.random.RandomState(rseed)
    A = make_spd_matrix(n_dim, random_state=rseed)
    x = rng.randint(-10, 10, n_dim)
    b = A @ x

    def f(x):
      return 0.5 * x.T @ A @ x - b @ x
    
    def grad_f(x):
      return A @ x - b

    def hessian_f(x):
      return A
    
    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = x
    self.A = A
    self.b = b

  def rosenbrock(self):

    def f(x):
      return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

    def grad_f(x):
      return np.array([-400*x[0]*(x[1] - x[0]**2) - 2*(1 - x[0]),
           200*(x[1] - x[0]**2)])

    def hessian_f(x):
      return np.array([[-400*(x[1] - 3*x[0]**2) + 2, -400*x[0]],
           [-400*x[0], 200]])
      
    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = np.array([1,1])

  def himmelblau(self):

    def f(x):
      return (x[0] ** 2 + x[1] - 11) ** 2 + (x[0] + x[1] ** 2 - 7) ** 2
    
    def grad_f(x):
      return np.array([4*x[0]*(x[0]**2 + x[1] - 11)+2*(x[0] + x[1]**2 - 7),
                       4*x[1]*(x[1]**2 + x[0] - 7)+2*(x[1] + x[0]**2 - 11)])
    
    def hessian_f(x):
      return np.array([[12*x[0]**2 + 4*x[1] - 42, 4*(x[1] + x[0])],
                       [4*(x[1] + x[0]), 12*x[1]**2 + 4*x[0] - 26]])
      
    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = np.array([[3,2], [-2.805118, 3.131312], [-3.779310, -3.283186], [3.584428, -1.848126]])

  def poly_1(self):

    def f(x):
      return ((x - 7)**2 * (x - 3)**2) / 4
    
    def grad_f(x):
      return (x - 7) * (x - 5) * (x - 3)
    
    def hessian_f(x):
      return 3 * x**2 - 30 * x + 71

    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = np.array([[3],[5],[7]])

  def poly_2(self):

    def f(x):
      return (x**2 * (x**2 - 16*x + 40)) / 4
    
    def grad_f(x):
      return x * (x - 2) * (x - 10)
    
    def hessian_f(x):
      return 3 * x**2 - 24 * x + 20

    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = np.array([[0],[2],[10]])

  def poly_3(self):

    def f(x):
      return (x * (3 * x**3 - 64 * x**2 + 414 * x - 648)) / 12
    
    def grad_f(x):
      return (x - 1) * (x - 6) * (x - 9)
    
    def hessian_f(x):
      return 3 * x**2 - 32 * x + 69

    self.f = f
    self.grad_f = grad_f
    self.hessian_f = hessian_f
    self.min_x = np.array([[1],[6],[9]])

<hr><h4 style="background-color: #ADB8FF;"><br>Optimising algorithm itself<br></h4><br>
<i>backtracking_alpha() implements the backtracking line search to find a suitable step length as described in the book. FR() implements the Fletcher Reeves nonlinear conjugate gradient method.<i>

In [ ]:
def backtracking_alpha(f, grad_f, xk, pk, alpha0=1, rho=0.95, c=1e-4):

  alpha = alpha0

  while not f(xk + alpha * pk) <= (f(xk) + c * alpha * grad_f(xk).T @ pk):
    alpha *= rho
  
  return alpha

def FR(x0, f, grad_f=None):

  conv_tol = 1e-8
  if grad_f == None:
    grad_f = approx_grad(f)
    conv_tol = 1e-6
  i = 0
  xk = x0
  pk = -grad_f(xk)

  while not stop_crit(grad_f, xk, x0, i, tol=conv_tol):

    xk1 = xk + backtracking_alpha(f, grad_f, xk, pk) * pk
    beta = (grad_f(xk1) @ grad_f(xk1)) / (grad_f(xk) @ grad_f(xk))
    pk = -grad_f(xk1) + beta * pk
    xk = xk1
    i += 1
 
  print(f"\nsearch terminated at iteration {i} | result: {xk}")
  return xk

<h3 style="background-color: #D3D92B;"><br>Testing on 5-10 variables, Quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [ ]:
rseed = [1,4,6,7,8]
quadratic_probs = []
for i in range(5):
  prob = Problem()
  prob.quadratic(10, rseed[i])
  quadratic_probs.append(prob)

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><b>Note:</b> After every test print out the resulsts. 
<br>For your convinience we implemented a function which will do it for you. Function can be used in case after running optimisation you return $x_{optimal}$, and if you have implemented your gradient approximation. Feel free to bring your adjustments.
<br> Additionaly print how many iterations your algotithm needed. You might also provide charts of your taste (if you want).
<p><i>Place for your additional comments and argumentation<i>

In [ ]:
def final_printout(x_0,x_optimal,x_appr,f,grad,args,tolerance):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_0: numpy 1D array, corresponds to initial point
    x_optimal: numpy 1D array, corresponds to optimal point, which you know, or have solved analytically
    x_appr: numpy 1D array, corresponds to approximated point, which your algorithm returned
    --------------------------------------------------------------------------------------------------------------
    f: function which takes 2 inputs: x (initial, optimal, or approximated)
                                      **args
       Function f returns a scalar output.
    --------------------------------------------------------------------------------------------------------------
    grad: function which takes 3 inputs: x (initial, optimal, or approximated), 
                                         function f,
                                         args (which are submitted, because you might need
                                              to call f(x,**args) inside your gradient function implementation). 
          Function grad approximates gradient at given point and returns a 1d np array.
    --------------------------------------------------------------------------------------------------------------
    args: dictionary, additional (except of x) arguments to function f
    tolerance: float number, absolute tolerance, precision to which, you compare optimal and approximated solution.
    """
    
    print(f'Initial x is :\t\t{x_0}')
    print(f'Optimal x is :\t\t{x_optimal}')
    print(f'Approximated x is :\t{x_appr}')
    print(f'Is close verificaion: \t{np.isclose(x_appr,x_optimal,atol=tolerance)}\n')
    f_opt = f(x_optimal,**args)
    f_appr = f(x_appr,**args)
    print(f'Function value in optimal point:\t{f_opt}')
    print(f'Function value in approximated point:   {f_appr}')
    print(f'Is close verificaion:\t{np.isclose(f_opt,f_appr,atol=tolerance)}\n')
    print(f'Gradient approximation in optimal point is:\n{grad(f,x_optimal,args)}\n')
    grad_appr = grad(f,x_appr,args)
    print(f'Gradient approximation in approximated point is:\n{grad_appr}\n')
    print(f'Is close verificaion:\n{np.isclose(grad_appr,np.zeros(grad_appr.shape),atol=tolerance)}')

In [ ]:
for i, prob in enumerate(quadratic_probs):
  print(f"Problem {i+1}: ")
  print("approximated gradient: ")
  FR(np.zeros(10), prob.f)
  print("\n exact gradient: ")
  FR(np.zeros(10), prob.f, prob.grad_f)
  print(f"\n actual minimum: {prob.min_x}\n")

Problem 1: 
approximated gradient: 

search terminated at iteration 654 | result: [ -5.00003438   1.00002037   1.99998906  -2.00000149  -1.00000396
   0.99998883  -5.00000031   4.99999505 -10.00000355   6.00000115]

 exact gradient: 

search terminated at iteration 2930 | result: [-5.0000002   1.00000004  2.00000014 -2.00000001 -0.99999999  1.00000001
 -5.00000001  4.99999997 -9.99999989  6.00000011]

 actual minimum: [ -5   1   2  -2  -1   1  -5   5 -10   6]

Problem 2: 
approximated gradient: 

search terminated at iteration 202 | result: [ 4.00007019 -4.99997688 -8.99996437 -1.99993588 -1.99998155  7.99998998
 -0.99994231 -3.00000223  3.00000139 -1.99999319]

 exact gradient: 

search terminated at iteration 868 | result: [ 4.00000017 -5.00000008 -8.99999984 -2.00000021 -2.00000035  8.
 -1.00000015 -2.9999998   2.99999965 -2.00000025]

 actual minimum: [ 4 -5 -9 -2 -2  8 -1 -3  3 -2]

Problem 3: 
approximated gradient: 

search terminated at iteration 1226 | result: [ 1.28542515e-06

<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

<h3 style="background-color: #D3D92B;"><br>Testing on functions of 1-2 variables, Non-quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

Problem rosenbrock: 

approximate gradient: 

search terminated at iteration 548 | result: [1.00000649 1.00001329]

exact gradient: 

search terminated at iteration 1037 | result: [0.99999979 0.99999958]

actual minimum: [1 1]

Problem himmelblau: 

approximate gradient: 

search terminated at iteration 523 | result: [2.99999955 2.00000087]

exact gradient: 

search terminated at iteration 769 | result: [-2.80511808  3.13131252]

actual minimum: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Problem poly_1: 

approximate gradient: 

search terminated at iteration 42 | result: [7.00000184]

exact gradient: 

search terminated at iteration 60 | result: [6.99999999]

actual minimum: [[3]
 [5]
 [7]]

Problem poly_2: 

approximate gradient: 

search terminated at iteration 338 | result: [9.9999998]

exact gradient: 

search terminated at iteration 1444 | result: [10.]

actual minimum: [[ 0]
 [ 2]
 [10]]

Problem poly_3: 

approximate gradient: 

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><i>Place for your additional comments and argumentation<i>

In [ ]:
prob = Problem()
prob.rosenbrock()
print(f"Problem rosenbrock: \n")
print("approximate gradient: ")
FR(np.array([1.2,1.2]), prob.f)
print("\nexact gradient: ")
FR(np.array([1.2,1.2]), prob.f, prob.grad_f)
print(f"\nactual minimum: {prob.min_x}")

prob = Problem()
prob.himmelblau()
print(f"\nProblem himmelblau: \n")
print("approximate gradient: ")
FR(np.array([0,0]), prob.f)
print("\nexact gradient: ")
FR(np.array([0,0]), prob.f, prob.grad_f)
print(f"\nactual minimum: {prob.min_x}")

prob = Problem()
prob.poly_1()
print(f"\nProblem poly_1: \n")
print("approximate gradient: ")
FR(np.array([2]), prob.f)
print("\nexact gradient: ")
FR(np.array([2]), prob.f, prob.grad_f)
print(f"\nactual minimum: {prob.min_x}")

prob = Problem()
prob.poly_2()
print(f"\nProblem poly_2: \n")
print("approximate gradient: ")
FR(np.array([1]), prob.f)
print("\nexact gradient: ")
FR(np.array([1]), prob.f, prob.grad_f)
print(f"\nactual minimum: {prob.min_x}")

prob = Problem()
prob.poly_3()
print(f"\nProblem poly_3: \n")
print("approximate gradient: ")
FR(np.array([7]), prob.f)
print("\nexact gradient: ")
FR(np.array([7]), prob.f, prob.grad_f)
print(f"\nactual minimum: {prob.min_x}")

Problem rosenbrock: 

approximate gradient: 

search terminated at iteration 548 | result: [1.00000649 1.00001329]

exact gradient: 

search terminated at iteration 1037 | result: [0.99999979 0.99999958]

actual minimum: [1 1]

Problem himmelblau: 

approximate gradient: 

search terminated at iteration 523 | result: [2.99999955 2.00000087]

exact gradient: 

search terminated at iteration 769 | result: [-2.80511808  3.13131252]

actual minimum: [[ 3.        2.      ]
 [-2.805118  3.131312]
 [-3.77931  -3.283186]
 [ 3.584428 -1.848126]]

Problem poly_1: 

approximate gradient: 

search terminated at iteration 42 | result: [7.00000184]

exact gradient: 

search terminated at iteration 60 | result: [6.99999999]

actual minimum: [[3]
 [5]
 [7]]

Problem poly_2: 

approximate gradient: 

search terminated at iteration 338 | result: [9.9999998]

exact gradient: 

search terminated at iteration 1444 | result: [10.]

actual minimum: [[ 0]
 [ 2]
 [10]]

Problem poly_3: 

approximate gradient: 

<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

<h3 style="background-color: #D3D92B;"><br>Template for teachers' tests<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Set up a template, how one can run your code<br></h4><br>
Template should include sceletons for:<ul>
    <li>custom function to optimise over </li> 
    <li>values initialisation to submit into otimising algorithm </li> 
    <li>optimiser function call</li> 
    <li>report print out call</li> </ul><br>
Provide descriptions and comments.